# 1. Setup

Import librarys and load database into juypiter

### Connect to data base

In [18]:
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd
import geopandas as gpd
from datetime import datetime
from sqlalchemy import text
from sqlalchemy import inspect
from sqlalchemy import create_engine
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="sydney"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn
    
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

srid = 4326

In [19]:
db, conn = pgconnect(credentials)

Connected successfully.


### Import data into python and data cleaning

In [162]:
SA2_Regions = gpd.read_file('SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
SA2_Regions = SA2_Regions[SA2_Regions['GCC_CODE21'] == "1GSYD"]

def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)
SA2_Regionsog = SA2_Regions.copy() 
SA2_Regions['geom'] = SA2_Regions['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid)) 
SA2_Regions.columns = map(str.lower, SA2_Regions.columns)

,sa2_code21,sa2_name21,chg_flag21,chg_lbl21,sa3_code21,sa3_name21,sa4_code21,sa4_name21,gcc_code21,gcc_name21,ste_code21,ste_name21,aus_code21,aus_name21,areasqkm21,loci_uri21,geometry,geom
28,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.41373 -33.46558, 151.41362 -33.4...",MULTIPOLYGON (((151.413733024921 -33.465580583...


In [163]:
SA2_Regions = SA2_Regions.drop(columns=['geometry', 'gcc_code21', 'gcc_name21', 'ste_code21', 'ste_name21', 'aus_code21', 'aus_name21'])
SA2_Regions.head(1)

,sa2_code21,sa2_name21,chg_flag21,chg_lbl21,sa3_code21,sa3_name21,sa4_code21,sa4_name21,areasqkm21,loci_uri21,geom
28,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.413733024921 -33.465580583...


Businesses dataset:

change the name of columns start with number, to match the SQL formate\n

set sa2_code as primary key, check if any row has empty primary key

In [109]:
businesses = pd.read_csv('Businesses.csv')

column_mapping = {
    '0_to_50k_businesses': 'businesses_0_50k',
    '50k_to_200k_businesses': 'businesses_50k_200k',
    '200k_to_2m_businesses': 'businesses_200k_2m',
    '2m_to_5m_businesses': 'businesses_2m_5m',
    '5m_to_10m_businesses': 'businesses_5m_10m',
    '10m_or_more_businesses': 'businesses_10m_or_more',
    'total_businesses': 'total_businesses'
}
businesses.rename(columns=column_mapping, inplace=True)

# Check for missing values in the 'sa2_code' column, as this is the primary key
missing_sa2_code = businesses['sa2_code'].isnull()
print (f"{businesses['sa2_code'].isnull().sum()} rows with empty primary key have been deleted")
businesses = businesses[~missing_sa2_code]
businesses.columns = map(str.lower, businesses.columns)

businesses.head(1)




0 rows with empty primary key have been deleted


,industry_code,industry_name,sa2_code,sa2_name,businesses_0_50k,businesses_50k_200k,businesses_200k_2m,businesses_2m_5m,businesses_5m_10m,businesses_10m_or_more,total_businesses
0,A,"Agriculture, Forestry and Fishing",101021007,Braidwood,136,92,63,4,0,0,296


In [23]:
businesses.dtypes

industry_code             object
industry_name             object
sa2_code                   int64
sa2_name                  object
businesses_0-50k           int64
businesses_50k-200k        int64
businesses_200k-2m         int64
businesses_2m-5m           int64
businesses_5m-10m          int64
businesses_10m_or_more     int64
total_businesses           int64
dtype: object

Income dataset:

In [96]:
income = pd.read_csv('Income.csv')

missing_sa2_code = income['sa2_code21'].isnull()
print (f"{income['sa2_code21'].isnull().sum()} rows with empty primary key have been deleted")
income = income[~missing_sa2_code]

income.dtypes


0 rows with empty primary key have been deleted


sa2_code21        int64
sa2_name         object
earners          object
median_age       object
median_income    object
mean_income      object
dtype: object

Population dataset:

In [110]:
population = pd.read_csv('Population.csv')

column_mapping = {
    '0-4_people': 'people_0_4',
    '5-9_people': 'people_5_9',
    '10-14_people': 'people_10_14',
    '15-19_people': 'people_15_19',
    '20-24_people': 'people_20_24',
    '25-29_people': 'people_25_29',
    '30-34_people': 'people_30_34',
    '35-39_people': 'people_35_39',
    '40-44_people': 'people_40_44',
    '45-49_people': 'people_45_49',
    '50-54_people': 'people_50_54',
    '55-59_people': 'people_55_59',
    '60-64_people': 'people_60_64',
    '65-69_people': 'people_65_69',
    '70-74_people': 'people_70_74',
    '75-79_people': 'people_75_79',
    '80-84_people': 'people_80_84',
    '85-and-over_people': 'people_85_and_over',
    'total_people': 'total_people'
}
population.rename(columns=column_mapping, inplace=True)
population.dtypes


sa2_code               int64
sa2_name              object
people_0_4             int64
people_5_9             int64
people_10_14           int64
people_15_19           int64
people_20_24           int64
people_25_29           int64
people_30_34           int64
people_35_39           int64
people_40_44           int64
people_45_49           int64
people_50_54           int64
people_55_59           int64
people_60_64           int64
people_65_69           int64
people_70_74           int64
people_75_79           int64
people_80_84           int64
people_85_and_over     int64
total_people           int64
dtype: object

In [161]:
schools_future = gpd.read_file('catchments/catchments_future.shp')

schools_futureog = schools_future.copy() 
schools_future['geom'] = schools_future['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid)) 
schools_future = schools_future.drop(columns="geometry")
schools_future['ADD_DATE'] = schools_future['ADD_DATE'].apply(lambda x: datetime.strptime(str(x), "%Y%m%d") if x is not None else None)
schools_future.columns = map(str.lower, schools_future.columns)
schools_future.head(1)

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,geom
0,8416,HIGH_COED,Ku-ring-gai HS,2023-01-14,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((151.19848917708944 -33.5398987...


In [125]:
schools_primary = gpd.read_file('catchments/catchments_primary.shp')

schools_primaryog = schools_primary.copy() 
schools_primary['geom'] = schools_primary['geometry'].apply(lambda x: create_wkt_element(geom=x, srid=srid)) 
schools_primary = schools_primary.drop(columns="geometry")

schools_primary['ADD_DATE'] = schools_primary['ADD_DATE'].apply(lambda x: datetime.strptime(str(x), "%Y%m%d") if x is not None else None)

schools_primary.columns = map(str.lower, schools_primary.columns)

print(schools_primary.dtypes)

use_id                object
catch_type            object
use_desc              object
add_date      datetime64[ns]
kindergart            object
year1                 object
year2                 object
year3                 object
year4                 object
year5                 object
year6                 object
year7                 object
year8                 object
year9                 object
year10                object
year11                object
year12                object
priority              object
geom                  object
dtype: object


In [127]:
schools_secondary = gpd.read_file('catchments/catchments_secondary.shp')

schools_secondaryog = schools_secondary.copy() 
schools_secondary['geom'] = schools_secondary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid)) 
schools_secondary = schools_secondary.drop(columns="geometry")
schools_secondary['ADD_DATE'] = schools_secondary['ADD_DATE'].apply(lambda x: datetime.strptime(str(x), "%Y%m%d") if x is not None else None)

schools_secondary.columns = map(str.lower, schools_secondary.columns)
schools_secondary.dtypes

use_id                object
catch_type            object
use_desc              object
add_date      datetime64[ns]
kindergart            object
year1                 object
year2                 object
year3                 object
year4                 object
year5                 object
year6                 object
year7                 object
year8                 object
year9                 object
year10                object
year11                object
year12                object
priority              object
geom                  object
dtype: object

In [146]:
stops = pd.read_csv('Stops.csv')

# Check for missing values in the 'stop_id' column, as this is the primary key
missing_stop_id = stops['stop_id'].isnull()
print (f"{stops['stop_id'].isnull().sum()} rows with empty primary key have been deleted")
stops = stops[~missing_stop_id]
stops['geom'] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat)
stops = stops.drop(columns=['stop_lon', 'stop_lat'])
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
stops['geom'] = stops['geom'].apply(lambda x: str(x))
stops.head()

0 rows with empty primary key have been deleted


,stop_id,stop_code,stop_name,location_type,parent_station,wheelchair_boarding,platform_code,geom
0,200039,200039.0,"Central Station, Eddy Av, Stand A",NaN,200060,0,NaN,POINT (151.20666465471 -33.8822064874687)
1,200054,200054.0,"Central Station, Eddy Av, Stand D",NaN,200060,0,NaN,POINT (151.20699145565 -33.8820421431408)
2,200060,NaN,Central Station,1.0,NaN,0,NaN,POINT (151.206292455081 -33.8840842535493)
3,201510,NaN,Redfern Station,1.0,NaN,0,NaN,POINT (151.198866071817 -33.8916900512711)
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",NaN,201510,0,NaN,POINT (151.198881722942 -33.8933293130144)


In [121]:
polling = pd.read_csv('PollingPlaces2019.csv')

missing_FID = polling['FID'].isnull()
print(f"{missing_FID.sum()} rows with empty primary key have been deleted")
polling = polling[~missing_FID]

missing_geom = polling['the_geom'].isnull()
print(f"{missing_geom.sum()} rows with empty geom have been deleted")
polling = polling[~missing_geom]

polling['the_geom'] = polling['the_geom'].apply(lambda x: Point(float(x.split()[1].strip("(")), float(x.split()[2].strip(")"))))

polling['the_geom'] = polling['the_geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

polling = polling.drop(columns=['latitude', 'longitude'])

polling['premises_post_code'] = polling['premises_post_code'].astype(int)
polling.columns = map(str.lower, polling.columns)

polling.dtypes

0 rows with empty primary key have been deleted
140 rows with empty geom have been deleted


fid                            object
state                          object
division_id                     int64
division_name                  object
polling_place_id                int64
polling_place_type_id           int64
polling_place_name             object
premises_name                  object
premises_address_1             object
premises_address_2             object
premises_address_3             object
premises_suburb                object
premises_state_abbreviation    object
premises_post_code              int64
the_geom                       object
dtype: object

### load dataset into SQL data base

create schema for each table

In [71]:
# Connect to the database and specify the schema
conn.execute('''
CREATE SCHEMA IF NOT EXISTS sydney;
SET search_path TO sydney;
''')


query(conn, "select PostGIS_Version()")

,postgis_version
0,3.4 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [164]:
conn.execute('''
DROP TABLE IF EXISTS sydney.sa2_regions;
CREATE TABLE sydney.sa2_regions (
    sa2_code21 VARCHAR(20),
    sa2_name21 VARCHAR(100),
    chg_flag21 VARCHAR(2),
    chg_lbl21 VARCHAR(100),
    sa3_code21 VARCHAR(20),
    sa3_name21 VARCHAR(100),
    sa4_code21 VARCHAR(20),
    sa4_name21 VARCHAR(100),
    areasqkm21 NUMERIC,
    loci_uri21 VARCHAR(500),
    geom GEOMETRY(MULTIPOLYGON, 4326)
);
''')

conn.execute('''
DROP TABLE IF EXISTS sydney.businesses;
CREATE TABLE sydney.businesses (
    industry_code VARCHAR(20),
    industry_name VARCHAR(100),
    sa2_code INTEGER,
    sa2_name VARCHAR(100),
    businesses_0_50k INTEGER,
    businesses_50k_200k INTEGER,
    businesses_200k_2m INTEGER,
    businesses_2m_5m INTEGER,
    businesses_5m_10m INTEGER,
    businesses_10m_or_more INTEGER,
    total_businesses INTEGER
);
''')

conn.execute('''
DROP TABLE IF EXISTS sydney.income;
CREATE TABLE sydney.income (
    sa2_code21 INTEGER,
    sa2_name VARCHAR(100),
    earners VARCHAR(100),
    median_age VARCHAR(100),
    median_income VARCHAR(100),
    mean_income VARCHAR(100)
);
''')

conn.execute('''
DROP TABLE IF EXISTS sydney.population;
CREATE TABLE sydney.population (
    sa2_code INTEGER,
    sa2_name VARCHAR(100),
    people_0_4 INTEGER,
    people_5_9 INTEGER,
    people_10_14 INTEGER,
    people_15_19 INTEGER,
    people_20_24 INTEGER,
    people_25_29 INTEGER,
    people_30_34 INTEGER,
    people_35_39 INTEGER,
    people_40_44 INTEGER,
    people_45_49 INTEGER,
    people_50_54 INTEGER,
    people_55_59 INTEGER,
    people_60_64 INTEGER,
    people_65_69 INTEGER,
    people_70_74 INTEGER,
    people_75_79 INTEGER,
    people_80_84 INTEGER,
    people_85_and_over INTEGER,
    total_people INTEGER
);
''')

conn.execute('''
DROP TABLE IF EXISTS sydney.stops;
CREATE TABLE sydney.stops (
    stop_id VARCHAR(100),
    stop_code FLOAT,
    stop_name VARCHAR(100),
    location_type FLOAT,
    parent_station VARCHAR(100),
    wheelchair_boarding INTEGER,
    platform_code VARCHAR(100),
    geom GEOMETRY(POINT,4326)
);
''')

conn.execute("""
DROP TABLE IF EXISTS sydney.schools_future;
CREATE TABLE sydney.schools_future (
    USE_ID VARCHAR(100),
    CATCH_TYPE VARCHAR(100),
    USE_DESC VARCHAR(100),
    ADD_DATE TIMESTAMP,
    KINDERGART INTEGER,
    YEAR1 INTEGER,
    YEAR2 INTEGER,
    YEAR3 INTEGER,
    YEAR4 INTEGER,
    YEAR5 INTEGER,
    YEAR6 INTEGER,
    YEAR7 INTEGER,
    YEAR8 INTEGER,
    YEAR9 INTEGER,
    YEAR10 INTEGER,
    YEAR11 INTEGER,
    YEAR12 INTEGER,
    geom GEOMETRY(MULTIPOLYGON,4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS sydney.schools_primary;
CREATE TABLE sydney.schools_primary (
    USE_ID VARCHAR(100),
    CATCH_TYPE VARCHAR(100),
    USE_DESC VARCHAR(100),
    ADD_DATE TIMESTAMP,
    KINDERGART VARCHAR(100),
    YEAR1 VARCHAR(100),
    YEAR2 VARCHAR(100),
    YEAR3 VARCHAR(100),
    YEAR4 VARCHAR(100),
    YEAR5 VARCHAR(100),
    YEAR6 VARCHAR(100),
    YEAR7 VARCHAR(100),
    YEAR8 VARCHAR(100),
    YEAR9 VARCHAR(100),
    YEAR10 VARCHAR(100),
    YEAR11 VARCHAR(100),
    YEAR12 VARCHAR(100),
    PRIORITY VARCHAR(100),
    geom GEOMETRY(MULTIPOLYGON,4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS sydney.schools_secondary;
CREATE TABLE sydney.schools_secondary (
    USE_ID VARCHAR(100),
    CATCH_TYPE VARCHAR(100),
    USE_DESC VARCHAR(100),
    ADD_DATE TIMESTAMP,
    KINDERGART VARCHAR(100),
    YEAR1 VARCHAR(100),
    YEAR2 VARCHAR(100),
    YEAR3 VARCHAR(100),
    YEAR4 VARCHAR(100),
    YEAR5 VARCHAR(100),
    YEAR6 VARCHAR(100),
    YEAR7 VARCHAR(100),
    YEAR8 VARCHAR(100),
    YEAR9 VARCHAR(100),
    YEAR10 VARCHAR(100),
    YEAR11 VARCHAR(100),
    YEAR12 VARCHAR(100),
    PRIORITY VARCHAR(100),
    geom GEOMETRY(MULTIPOLYGON,4326)
);
""")


conn.execute("""
CREATE SCHEMA IF NOT EXISTS sydney;
DROP TABLE IF EXISTS sydney.polling;
CREATE TABLE sydney.polling (
    FID VARCHAR(100),
    state VARCHAR(100),
    division_id INTEGER,
    division_name VARCHAR(100),
    polling_place_id INTEGER,
    polling_place_type_id INTEGER,
    polling_place_name VARCHAR(100),
    premises_name VARCHAR(100),
    premises_address_1 VARCHAR(100),
    premises_address_2 VARCHAR(100),
    premises_address_3 VARCHAR(100),
    premises_suburb VARCHAR(100),
    premises_state_abbreviation VARCHAR(100),
    premises_post_code INTEGER,
    the_geom GEOMETRY(POINT,4326)
);
""")



In [144]:
businesses.to_sql("businesses", con=conn, schema="sydney", if_exists='append', index=False)
income.to_sql("income", con=conn, schema="sydney", if_exists='append', index=False)
population.to_sql("population", con=conn, schema="sydney", if_exists='append', index=False)
stops.to_sql("stops", con=conn, schema="sydney", if_exists='append', index=False)


373

In [165]:

SA2_Regions.to_sql("sa2_regions", con=conn, schema="sydney", if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
schools_future.to_sql("schools_future", con=conn, schema="sydney", if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
schools_primary.to_sql("schools_primary", con=conn, schema="sydney", if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
schools_secondary.to_sql("schools_secondary", con=conn, schema="sydney", if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
polling.to_sql('polling', con=conn, schema="sydney", if_exists='append', index=False, dtype={'the_geom': Geometry('POINT', srid)})


790

In [130]:
sql = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'sydney'
"""
query(conn, sql)

,table_name
0,geography_columns
1,geometry_columns
2,spatial_ref_sys
3,businesses
4,population
5,sa2_regions
6,income
7,stops
8,schools_future
9,schools_primary
